In [1]:
import os
import sys
import numpy as np
import pandas as pd
import torch

sys.path.append('..')
from utils.utils_ctc import setup, preprocess, MyDataLoader, get_dataloader

In [2]:
path = '/mnt/aoni04/jsakuma/data/sota'
DENSE_FLAG = False
ELAN_FLAG = True
TARGET_TYPE = False

seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

dataloaders_dict = get_dataloader(path, DENSE_FLAG, ELAN_FLAG, TARGET_TYPE)

 13%|█▎        | 13/100 [00:00<00:06, 13.79it/s]

/mnt/aoni04/jsakuma/data/sota/TEXT/20171202141840.vad.txt


100%|██████████| 12/12 [00:01<00:00,  7.83it/s]


turn taking timing: 2512
overlap: 392
turn taking timing: 342
overlap: 49


In [3]:
input_batch, target_batch = dataloaders_dict['val'].next()

In [65]:
input_batch, target_batch = dataloaders_dict['val'].next()

In [66]:
input_batch['voiceA'][0][0].shape

(45, 128)

In [67]:
input_batch['phonemeA'][0][0].shape

(45,)

In [68]:
target_batch['U'][0][0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [9]:
u_true, u_pred = np.array([]), np.array([])
for input_batch, target_batch in dataloaders_dict['val']:
    u = target_batch['U'][0]
    for i in range(len(u)):
        u_true = np.append(u_true, u[i])

In [10]:
len(u_true)

88842

In [11]:
u_true, u_pred = np.array([]), np.array([])
for input_batch, target_batch in dataloaders_dict['train']:
    u = target_batch['U'][0]
    for i in range(len(u)):
        u_true = np.append(u_true, u[i])
        
print(len(u_true))

550748


In [19]:
path = '/mnt/aoni04/jsakuma/data/sota'
dense_flag = False
elan_flag = True

seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

file_id = [file.replace('.feature.csv', '') for file in sorted(os.listdir(os.path.join(path, 'feature_50_p')))]
train_num = 100
feat_train = setup(path, file_id[:train_num], dense_flag=dense_flag, elan_flag=elan_flag)
feat_val = setup(path, file_id[train_num:], dense_flag=dense_flag, elan_flag=elan_flag)

 12%|█▏        | 12/100 [00:01<00:10,  8.15it/s]

/mnt/aoni04/jsakuma/data/sota/TEXT/20171202141840.vad.txt


100%|██████████| 12/12 [00:01<00:00,  8.03it/s]


In [29]:
target_type=False
input_dict_train, target_dict_train = preprocess(feat_train, target_type=target_type, phase='train')
input_dict_val, target_dict_val = preprocess(feat_val, target_type=target_type, phase='val')

turn taking timing: 2512
overlap: 392
turn taking timing: 342
overlap: 49


In [33]:
cnt=0
for i in range(12):
    for arr in target_dict_val['U'][i]:
        cnt+=len(arr)
    
print(cnt)

88842


In [34]:
len(target_dict_val['y'])

12

In [21]:
feat = feat_val
i=0

u = (1 - feat['feature'][i]['utter_A'].values) * (1 - feat['feature'][i]['utter_B'].values)  # 非発話度真値 AもBもOFFなら u=1
ua = feat['u'][i]['u_pred'].values
ub = feat['u'][i]['u_B_pred'].values
u_pred = np.min(np.stack([ua, ub], axis=1), axis=1)

In [22]:
len(u)

5502

In [23]:
s = np.asarray([0]+list(u[:-1]-u[1:]))
df = pd.DataFrame({'u': u, 's': s})

df.head(10)

,u,s
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
5,1.0,0.0
6,1.0,0.0
7,1.0,0.0
8,1.0,0.0
9,1.0,0.0


In [24]:
idxs = np.where(s==1)[0]

In [25]:
def make_pack(X, idxs):
    pack = []
    pre=0
    for i in idxs:
        pack.append(X[pre:i])
        pre = i

    pack.append(X[pre:])
    return pack

In [26]:
pack = make_pack(u, idxs)

In [27]:
u_true = np.array([])
for i in range(len(pack)):
    u_true = np.append(u_true, pack[i])

In [28]:
len(u_true)

5502

In [ ]:
input_dict = {'voiceA': [], 'voiceB': [], 'img': [], 'phonemeA': [], 'phonemeB': [], 'u': []}
    target_dict = {'y': [], 'action': []}

    X, PA, PB, U, U_pred, Y, Action = [], [], [], [], [], [], []
    OC, YC = 0, 0
    threshold = 0.8
    for i in range(len(feat)):
        u = (1 - feat['feature'][i]['utter_A'].values) * (1 - feat['feature']['utter_B'].values)  # 非発話度真値 AもBもOFFなら u=1
        ua = feat['u'][i]['u_pred'].values
        ub = feat['u'][i]['u_B_pred'].values
        u_pred = np.min(np.stack([ua, ub], axis=1), axis=1)

        # system の顔向き情報の入れ方
        # 画像特徴量も使う際に使用
        if not target_type:
            target = feat['feature'][i]['target'].map(lambda x: 0 if x == 'A' else 1).values
            target = target.reshape(len(target), 1)
        else:
            target = make_target(feat['feature'][i])

        img = feat['img'][i].values
        img = np.append(target, img, axis=1)

        # 教師ラベル
        y = feat['feature'][i]['action'].map(lambda x: threshold if x in ['Active','Passive'] else 0)
        action = feat['feature'][i]['action'].map(lambda x: 1 if x == 'Passive' else 0)
        action_c = feat['feature'][i]['action'].map(lambda x: 1 if x in ['Active-Continue','Passive-Continue'] else 0).values
        y[0] = 0.  # start は予測不可
        action[0] = 0
        y = np.asarray(y)
        action = np.asarray(action)
        YC += len(y[y > 0])
        over_flg = False
        start_i = 0
        act_tmp = 0
        for j in range(len(y)-1):
            if y[j] > 0 and u[j] == 0:
                y[j] = 0
                action[j] = 0
                act_tmp = action[j]
                OC += 1
                start_i = j
                over_flag = True

            if u[j] == 0 and u[j+1] == 1:
                if over_flg:
                    if abs(j-start_i) > 40:
                        over_flag = False
                        action_c[start_i:j+1] = 0
                    else:
                        y[j+2] = threshold
                        action[j+2] = act_tmp
                        act_tmp = 0
                        u[j+2] = 1
                        action_c[start_i:j+1] = 0
                        action[start_i:j+1] = 0
#                         action_c[j+3:j+3+(j-start_i)] = 1
                        over_flg = False

        # system の発話中ラベル
        if phase == 'train':  # sys発話中は u(t) を下げる
            u[action_c == 1] = 0.
            u_pred[action_c == 1] = 0.
        # 評価時は下げない,ただ長いシステム発話だとu(t)=1がずっと
        # 続いて１字遅れ系は必ずthreshold超えてしまうので u=1 区間にmax値を設ける
        else:
            u[action_c == 1] = 1.
#             u_pred[action_c==1] = 1.
#             u = u_t_maxcut(u, max_frame=60)
            u, u_pred = u_t_maxcut(u, u_pred, max_frame=60)

        # 非発話区間全体のアクションラベルを統一する
        flg = False
        start = 0
        end = 0
        act_tmp = 0
        for j in range(len(u)-1):
            if u[j] == 0 and u[j+1] == 1:
                start = j+1
            if y[j] > 0:
                flg = True
                act_tmp = action[j]
            if u[j] == 1 and u[j+1] == 0:
                end = j+1
                if flg:
                    action[start:end] = act_tmp
                    flg = False

        x, pa_, pb_, u, u_pred, y, action = make_pack(x, pa[i], pb[i], u, u_pred, y, action)

        X.append(x)
        PA.append(pa_)
        PB.append(pb_)
        U.append(u)
        U_pred.append(u_pred)
        Y.append(y)
        Action.append(action)

    print('turn taking timing: {}'.format(YC))
    print('overlap: {}'.format(OC))

    return input_dict, target_dict


In [11]:
import pandas as pd
df = pd.DataFrame(feature_dict)

array([[0.8877424 , 0.90143412],
       [0.94236445, 0.97174013],
       [0.98102772, 0.99062192],
       ...,
       [0.00938658, 0.94423807],
       [0.02363596, 0.91520423],
       [0.06521004, 0.94034523]])

In [5]:
spec_list = os.listdir('/mnt/aoni04/jsakuma/data/sota/spec/')

In [12]:
import glob
PATH='/mnt/aoni04/jsakuma/data/sota/'
img_middle_feature_files = sorted(glob.glob(os.path.join(PATH, 'spec/*npy')))

In [13]:
img_middle_feature_files

['/mnt/aoni04/jsakuma/data/sota/spec/20171201200850.Anpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171201200850.Bnpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171201201416.Anpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171201201416.Bnpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171201202038.Anpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171201202038.Bnpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171201202722.Anpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171201202722.Bnpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171202131909.Anpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171202131909.Bnpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171202132408.Anpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171202132408.Bnpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171202133148.Anpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171202133148.Bnpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171202133728.Anpy.npy',
 '/mnt/aoni04/jsakuma/data/sota/spec/20171202133728.Bnp

In [16]:
from tqdm import tqdm
for path in tqdm(img_middle_feature_files):
    if 'Anpy' in path:
        path2 = path.replace('Anpy', 'A')
        os.rename(path, path2) 
    elif 'Bnpy' in path:
        path2 = path.replace('Bnpy', 'B')
        os.rename(path, path2) 

100%|██████████| 224/224 [00:00<00:00, 1988.09it/s]


In [7]:
for i in spec_list:
    if '20200222174104' in i:
        print(i)

20200222174104.B.npy
20200222174104.A.npy


In [ ]:
    X_train, PA_train, PB_train, u_train, up_train, y_train, action_train = preprocess(feat_train, target_type=target_type, phase='train')
    X_val, PA_val, PB_val, u_val, up_val, y_val, action_val = preprocess(feat_val, target_type=target_type, phase='val')

    train_loader = MyDataLoader(X_train, PA_train, PB_train, u_train, up_train, y_train, action_train, shuffle=True, batch_size=batch_size)
    val_loader = MyDataLoader(X_val, PA_val, PB_val, u_val, up_val, y_val, action_val, shuffle=False, batch_size=batch_size)

    dataloaders_dict = {"train": train_loader, "val": val_loader}
    return dataloaders_dict

